In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

In [2]:
dws = r'E:\REMM\REMM_2020_Data_Inputs\DWSEmployment2019.gdb\DWSEmployment2019_located'
dws_sdf = pd.DataFrame.spatial.from_featureclass(dws)

In [3]:
# Recode dws data
dws_sdf['Tag'] = ""
dws_sdf['NAICS']= dws_sdf['NAICS'].astype(str) 

# industry?
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('3', na=False), 'Tag'] = "industrial"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('562', na=False), 'Tag'] = "industrial"

# wholesale?
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('48', na=False), 'Tag'] = "commercial"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('49', na=False), 'Tag'] = "commercial"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('423', na=False), 'Tag'] = "commercial"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('424', na=False), 'Tag'] = "commercial"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('425', na=False), 'Tag'] = "commercial"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('423', na=False), 'Tag'] = "commercial"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('424', na=False), 'Tag'] = "commercial"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('425', na=False), 'Tag'] = "commercial"

dws_sdf.loc[dws_sdf['NAICS'].str.startswith('51', na=False), 'Tag'] = "office"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('52', na=False), 'Tag'] = "office"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('53', na=False), 'Tag'] = "office"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('54', na=False), 'Tag'] = "office"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('55', na=False), 'Tag'] = "office"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('561', na=False), 'Tag'] = "office"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('81', na=False), 'Tag'] = "office"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('92', na=False), 'Tag'] = "office"

dws_sdf.loc[dws_sdf['NAICS'].str.startswith('61', na=False), 'Tag'] = "education"

dws_sdf.loc[dws_sdf['NAICS'].str.startswith('62', na=False), 'Tag'] = "healthcare"

dws_sdf.loc[dws_sdf['NAICS'].str.startswith('71', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('111', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('238', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('441', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('442', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('443', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('444', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('445', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('446', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('447', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('448', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('451', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('452', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('453', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('454', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('721', na=False), 'Tag'] = "retail"
dws_sdf.loc[dws_sdf['NAICS'].str.startswith('722', na=False), 'Tag'] = "retail"

# # check result
# dws_sdf[dws_sdf['Tag'] == 'IndustryDWS']
print(dws_sdf['Tag'].value_counts())

office        17238
retail        11106
healthcare     4653
commercial     3528
industrial     2083
               1566
education      1117
Name: Tag, dtype: int64


In [4]:
dws_sdf.columns

Index(['OBJECTID', 'primary_key', 'input_street', 'input_zone', 'x', 'y',
       'score', 'locator', 'matchAddress', 'standardizedAddress',
       'addressGrid', 'message', 'YRQTR', 'Legal_Name', 'DBA', 'PHYSSADD',
       'PHYSSADD2', 'PHYS_CITY', 'PHYS_ST', 'PHYS_ZIP', 'NAICS', 'OWN',
       'COUNTY', 'July', 'LAT', 'LONG', 'MPO_ID', 'REMMSectorID', 'Located',
       'Method', 'PHYSSADD_corrected', 'PHYS_ZIP_corrected', 'SHAPE', 'Tag'],
      dtype='object')

In [5]:
# export
dws_sdf = dws_sdf[['primary_key','Legal_Name','x','y','PHYS_CITY', 'NAICS','July', 'Tag', 'SHAPE']].copy()
dws_sdf.columns = ['unique_id','Legal_Name','x','y','PHYS_CITY', 'NAICS','July', 'Tag', 'SHAPE']
dws_new = r'E:\REMM\REMM_2020_Data_Inputs\dws_2019_with_industry_tag'
dws_sdf.spatial.to_featureclass(location=dws_new)

'E:\\REMM\\REMM_2020_Data_Inputs\\dws_2019_with_industry_tag.shp'